In [1]:
import cv2
import pandas as pd
import numpy as np
import pytesseract
from Classes.line import Line
from Classes.rectangle import Rectangle
from Utils.print_rectangle import print_rectangles
from Utils.filter_rectangles import filter_rectangles
from Utils.get_maximum_intersecting_column import get_maximum_intersecting_column
from Utils.sort_rows import sort_rows


img = cv2.imread('Sem 2.png')
dilated = cv2.dilate(img, kernel=np.ones((5, 5), np.uint8), iterations=1)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 200, 200, apertureSize=3)
cv2.imwrite("dilated.jpg", dilated)
cv2.imwrite("gray.jpg", gray)
cv2.imwrite("edges.jpg", edges)

True

In [2]:
lsd = cv2.createLineSegmentDetector(cv2.LSD_REFINE_ADV, sigma_scale=0.35)
dlines = lsd.detect(edges)
lines = [Line(x0, y0, x1, y1) for x0, y0, x1, y1 in dlines[0][:, 0]]
imgCopy = img.copy()
for dline in dlines[0]:
    x0 = int(round(dline[0][0]))
    y0 = int(round(dline[0][1]))
    x1 = int(round(dline[0][2]))
    y1 = int(round(dline[0][3]))
    cv2.putText(imgCopy, "(" + str(x0) + ", " + str(y0) + ")", (x0, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(imgCopy, "(" + str(x1) + ", " + str(y1) + ")", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.line(imgCopy, (x0, y0), (x1, y1), (0, 255, 0), 2)

cv2.imwrite("lsd.jpg", imgCopy)

True

In [3]:
tableStructure = np.zeros(img.shape, dtype=np.uint8)
print(img.shape)

for line in lines:
  coordinates = line.coordinates_as_int()
  cv2.line(tableStructure, coordinates[0], coordinates[1], (255, 255, 255), 1)

(322, 1680, 3)


In [4]:
kernel = np.ones((5, 5), np.uint8)
tableStructure = cv2.dilate(tableStructure, kernel, iterations=1)
cv2.imwrite("tableStructure.jpg", tableStructure)

True

In [5]:
table_structure = cv2.cvtColor(tableStructure, cv2.COLOR_RGB2GRAY)
contours, hierarchy = cv2.findContours(table_structure, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
count = 0

rectangles = [] 

for c in contours:
  x, y, w, h = cv2.boundingRect(c)
  rectangles.append(Rectangle(x, y, w, h))

print_rectangles(rectangles, table_structure)

In [6]:
from PIL import Image
fcont = cv2.boundingRect(contours[-10])
x, y, w, h = fcont
print(x, y, h, w)

newImg = gray[y:y+h, x:x+w]
print(newImg.shape)
print(pytesseract.image_to_string(newImg))
Image.fromarray(newImg)


973 10 25 149
(25, 149)
PM-2:55:PM



In [7]:
filtered_rect = filter_rectangles(rectangles)

print(filtered_rect)

[<Classes.rectangle.Rectangle object at 0x000001E24CC25E50>, <Classes.rectangle.Rectangle object at 0x000001E24CC269C0>, <Classes.rectangle.Rectangle object at 0x000001E24CC25F40>, <Classes.rectangle.Rectangle object at 0x000001E24CC27200>, <Classes.rectangle.Rectangle object at 0x000001E24CC27FE0>, <Classes.rectangle.Rectangle object at 0x000001E24CC27710>, <Classes.rectangle.Rectangle object at 0x000001E265827230>, <Classes.rectangle.Rectangle object at 0x000001E265827140>, <Classes.rectangle.Rectangle object at 0x000001E265826930>, <Classes.rectangle.Rectangle object at 0x000001E265826AB0>, <Classes.rectangle.Rectangle object at 0x000001E2658268A0>, <Classes.rectangle.Rectangle object at 0x000001E2658272C0>, <Classes.rectangle.Rectangle object at 0x000001E265827290>, <Classes.rectangle.Rectangle object at 0x000001E2658272F0>, <Classes.rectangle.Rectangle object at 0x000001E265827170>, <Classes.rectangle.Rectangle object at 0x000001E265826D20>, <Classes.rectangle.Rectangle object at 

In [141]:
# By now, all the rectangles are the ones which we care about
# the rectangles need not be always ordered but they are sorted by their y coordinates first and then their x coordinates

In [8]:
rows = sort_rows(filtered_rect, img.shape[0]//20)
    

In [9]:
# row[0] is the first row: presents the columns
columns = rows[0]


df = [[] for _ in range(len(rows)-1)]
for col in columns:
    for i in range(1, len(rows)):
        row = rows[i]
        intersecting_column = get_maximum_intersecting_column(row, col)
        df[i-1].append(row[intersecting_column].get_text(gray))


pd.DataFrame(df, columns=[col.get_text(gray) for col in columns])

,Day Name,8:0:AM-8:55:AM.,9:0:AM-9:55:AM.,10:0:AM-10:55:AM,11:0:AM-11:55:AM,12:0:PM-12:55:PM,2:0:PM-2:55:PM.,3:0:PM-3:55:PM.,4:0:PM-4:55:PM.,5:0:PM-5:55:PM.
0,Mon,cS11002\nNR422,cS11002\nNR422,EC29201\nIn the Department,EC29201\nIn the Department,EC29201\nIn the Department,,£C21201\nNR311,£C21201\nNR311,
1,Tue,,,,,cS11002\nNR422,£C21201\nNR311,£C21201\nNR311,,
2,Wed,,,cy11003\nNR111,cS11002\nNR422,MA11004\nNR111,,,,
3,Thur,,cy11003\nNR111,,MA11004\nNR111,,,,,
4,Fri,,MA11004\nNR111,MA11004\nNR111,cy11003\nNR111,cy11003\nNR111,cy19003\nIn the Department,cy19003\nIn the Department,cy19003\nIn the Department,
